In [ ]:
# Import required libraries
from bs4 import BeautifulSoup
import requests
import csv
import time
import smtplib
from datetime import datetime

In [ ]:
# Amazon product URL
URL = "https://www.amazon.com.tr/Casio-F-91W-1Dg-Erkek-Kol-Saati/dp/B000J34HN4"

# HTTP headers
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:110.0) Gecko/20100101 Firefox/110.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.5",
    "DNT": "1",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1"
}

In [ ]:
# Function to check the product price
def check_price():
    response = requests.get(URL, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    # Extract product title
    title = soup.find(id="productTitle")
    title = title.get_text().strip()

    # Extract product price
    price_element = soup.find("span", {"class": "a-price-whole"})
    price = float(price_element.get_text().replace(".", "").replace(",", "."))

    return title, price

In [ ]:
# Function to log the price to a CSV file
def log_price_to_csv(title, price):
    filename = "PRICE.csv"
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Fixed import here
    data = [timestamp, title, price]

    # Check if the file exists, if not create it and write the header
    try:
        with open(filename, "x", newline="", encoding="UTF8") as f:
            writer = csv.writer(f)
            writer.writerow(["Timestamp", "Product Title", "Price"])
    except FileExistsError:
        pass  # File already exists, no need to write the header again

    # Append the price data to the CSV file
    with open(filename, "a", newline="", encoding="UTF8") as f:
        writer = csv.writer(f)
        writer.writerow(data)

In [ ]:
# Function to send email
def send_email(price, title):
    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.ehlo()
    server.login('senders_mail@gmail.com', 'senders_password')  # Replace with your email credentials

    subject = f"Price Alert: {title} is now {price} TRY!"
    body = f"The product '{title}' is now priced at {price} TRY. Check it out here: {URL}"

    msg = f"Subject: {subject}\n\n{body}"

    server.sendmail(
        'senders_mail@gmail.com',
        'your_email@gmail.com',  # Replace with your email
        msg
    )
    server.close()

    print("Email sent successfully!")

In [ ]:
# Function to monitor price and check if it crosses threshold
def monitor_price(threshold_price):
    while True:
        title, price = check_price()
        if price and price < threshold_price:
            send_email(price, title)  # Send an email if price is below the threshold
        log_price_to_csv(title, price) 
        print(f"Logged price for {title}: {price}")
        time.sleep(3600)  # Wait for 1 hour before checking again

In [ ]:
# Run the monitoring function with a threshold price
threshold_price = 300  # Target price
monitor_price(threshold_price)